In [2]:
from google.cloud import bigquery
import pandas as pd
from dotenv import load_dotenv
import os
from sklearn.cluster import KMeans
import numpy as np4
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

# Load environment variables from .env file
load_dotenv()

# Now you can use the environment variable
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")


# Create a BigQuery client
client = bigquery.Client()

# Your BigQuery query
query = """
SELECT * FROM `big-mamma-project.Habits_tables.orders_analysis_final5`
"""

# Run the query and convert it to a pandas dataframe
orders = client.query(query,location="EU").to_dataframe()

# Now df is a pandas DataFrame containing the results of the query
orders

,order_id,site_id,site_name,owner_id,owner_name,ticket_date,time_of_close_and_paid,time_of_opening,mains,order,...,Group,Booking,Privatisation,CnC,AFTERNOON,LEMONADE,LUNCH,DINNER,BREAKFAST_1,PRIVATISATION_1
0,A742259.68582,13,Circolo Popolare,212614.0,Giuseppe R,2023-01-28,2023-01-29 00:34:41.059000+00:00,2023-01-28 15:53:21.321000+00:00,14.0,0,...,0,0,0,0,1,0,0,0,0,0
1,A742259.138270,13,Circolo Popolare,401560.0,Jason Medina,2023-08-29,2023-08-29 22:08:47.323000+00:00,2023-08-29 22:05:51.245000+00:00,0.0,1,...,0,0,0,0,0,1,0,0,0,0
2,A742259.142309,13,Circolo Popolare,380210.0,Lorenzo Mirra,2023-09-10,2023-09-10 21:57:33.875000+00:00,2023-09-10 21:57:33.870000+00:00,0.0,-1,...,0,0,0,0,0,1,0,0,0,0
3,A713532.46689,13,Circolo Popolare,212614.0,Giuseppe R,2022-05-16,2022-05-16 11:27:29.458000+00:00,2022-05-16 11:22:37.015000+00:00,0.0,0,...,0,0,0,0,0,0,1,0,0,0
4,A742259.152128,13,Circolo Popolare,351038.0,Marco Lombardi,2023-10-09,2023-10-09 21:35:06.327000+00:00,2023-10-09 18:58:36.610000+00:00,12.0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
791907,A742259.89949,13,Circolo Popolare,380203.0,Ilaria Di Rocco,2023-04-07,2023-04-07 18:47:55.703000+00:00,2023-04-07 18:10:48.351000+00:00,1.0,1,...,0,0,0,0,0,0,0,1,0,0
791908,A91652.88058,13,Circolo Popolare,188970.0,Florentina,2021-11-12,2021-11-12 22:10:48.144000+00:00,2021-11-12 20:29:15.368000+00:00,2.0,1,...,0,0,0,0,0,0,0,1,0,0
791909,A742259.6889,13,Circolo Popolare,287933.0,Mario,2022-07-26,2022-07-26 12:10:43.177000+00:00,2022-07-26 11:34:25.472000+00:00,3.0,1,...,0,0,0,0,0,0,1,0,0,0
791910,A742259.7194,13,Circolo Popolare,212614.0,Giuseppe R,2022-07-26,2022-07-26 21:39:15.677000+00:00,2022-07-26 20:00:14.180000+00:00,3.0,1,...,0,1,0,0,0,0,0,1,0,0


In [3]:
orders.dtypes

order_id            object
site_id              Int64
site_name           object
owner_id           float64
owner_name          object
                    ...   
LEMONADE             Int64
LUNCH                Int64
DINNER               Int64
BREAKFAST_1          Int64
PRIVATISATION_1      Int64
Length: 66, dtype: object

In [31]:
import pandas as pd
from scipy import stats


# Select numerical columns only 


# Calculate Z-scores for each numerical column
z_scores = np.abs((orders[['covers','actual_duration']] - orders[['covers','actual_duration']].mean()) / orders[['covers','actual_duration']].std())

# Define a threshold for outliers (e.g., Z-score > 3, meaning values beyond 3 standard deviations)
threshold = 3

# Create a boolean mask for outliers
outlier_mask = (z_scores > threshold).any(axis=1)

# Remove outliers from the DataFrame
orders_no_outliers = orders[~outlier_mask]

# Keep only the outliers
orders_outliers_only = orders[outlier_mask]


In [34]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
orders.describe()

,site_id,owner_id,mains,order,covers,nb_saleslines,qty,payment_occurence,discount_amount,discount_occurence,account_discount_occurence,change_rate,tax_local,service_charge_local,revenue_excl_tax_local,tot_revenue_excl_tax_local,table_number,actual_duration,diff_seated_arrival_time,diff_opening_seated_time,Take_away_Bar,Breakfast,Extra_Bar,Extra_Food,Side,Beer,Wine_Bottle,Hot_Beverage,Alcohol,Mains_1,Wine_glass,Softs,Pizza,Desserts,Other,Cocktail,Pasta,Starters,Walkin,Group,Booking,Privatisation,CnC,AFTERNOON,LEMONADE,LUNCH,DINNER,BREAKFAST_1,PRIVATISATION_1
count,791912.0,791911.000000,791912.000000,791912.0,791912.0,791912.0,762197.000000,791912.0,762197.000000,791912.0,791912.0,791912.000000,791912.000000,791912.000000,791912.000000,791912.000000,7.902800e+05,791912.0,662125.000000,662127.000000,762197.0,762197.0,762197.0,762197.0,762197.0,762197.0,762197.0,762197.0,762197.0,762197.0,762197.0,762197.0,762197.0,762197.0,762197.0,762197.0,762197.0,762197.0,791912.0,791912.0,791912.0,791912.0,791912.0,791912.0,791912.0,791912.0,791912.0,791912.0,791912.0
mean,28.223681,268881.461014,2.604912,0.939577,2.622301,10.342682,10.215992,1.488284,1.339995,0.176339,0.609444,1.161838,16.243049,12.589388,86.471938,99.061326,1.187441e+02,89.290011,1.010812,7.847088,0.00002,0.030895,0.091116,0.140932,0.161759,0.290365,0.323234,0.332055,0.356715,0.429584,0.459671,0.665684,0.685312,0.938987,0.948601,1.206397,1.583703,1.703608,0.322124,0.009327,0.667929,0.000612,0.000008,0.069604,0.015374,0.349435,0.552935,0.012039,0.000612
std,22.844953,79345.445171,2.313641,0.273209,2339.246537,9.429242,12.008247,1.058216,6.657780,0.85011,5.235331,0.021184,74.946772,12.278132,583.934354,585.735557,4.868408e+03,180.643632,282.700380,282.671098,0.005249,0.334528,0.583135,0.719297,0.546521,0.897602,0.841353,0.988463,1.280347,0.935288,1.039123,1.140969,1.219079,1.582193,2.55212,1.968254,1.464005,2.069003,0.467291,0.096124,0.470957,0.02474,0.002753,0.254478,0.123036,0.476792,0.49719,0.109061,0.02474
min,12.0,116589.000000,-87.000000,-1.0,-999999.0,0.0,-837.000000,1.0,-324.000000,0.0,0.0,1.112694,-18801.765600,-414.130000,-150414.124400,-150414.124400,0.000000e+00,-41605.0,-584.000000,-203465.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,13.0,207205.000000,2.000000,1.0,2.0,6.0,6.000000,1.0,0.000000,0.0,0.0,1.146251,8.500100,7.010000,48.333300,55.200400,2.300000e+01,60.0,-4.000000,4.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,13.0,261097.000000,2.000000,1.0,2.0,8.0,8.000000,1.0,0.000000,0.0,0.0,1.162853,13.083300,10.370000,70.416600,80.520000,5.400000e+01,81.0,0.000000,9.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
75%,34.0,330294.000000,3.000000,1.0,3.0,12.0,12.000000,2.0,0.000000,0.0,0.0,1.176407,19.833332,15.450000,104.583200,119.806600,1.210000e+02,107.0,5.000000,13.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
max,83.0,485937.000000,230.000000,1.0,999999.0,715.0,5018.000000,45.0,756.000000,143.0,567.0,1.215136,34880.678900,795.750000,279045.431100,279045.431100,4.114070e+06,1735.0,203478.000000,77.000000,2.0,28.0,96.0,60.0,27.0,95.0,78.0,51.0,130.0,54.0,80.0,159.0,75.0,163.0,232.0,188.0,75.0,135.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [35]:
orders_outliers_only.describe()

,site_id,owner_id,mains,order,covers,nb_saleslines,qty,payment_occurence,discount_amount,discount_occurence,account_discount_occurence,change_rate,tax_local,service_charge_local,revenue_excl_tax_local,tot_revenue_excl_tax_local,table_number,actual_duration,diff_seated_arrival_time,diff_opening_seated_time,Take_away_Bar,Breakfast,Extra_Bar,Extra_Food,Side,Beer,Wine_Bottle,Hot_Beverage,Alcohol,Mains_1,Wine_glass,Softs,Pizza,Desserts,Other,Cocktail,Pasta,Starters,Walkin,Group,Booking,Privatisation,CnC,AFTERNOON,LEMONADE,LUNCH,DINNER,BREAKFAST_1,PRIVATISATION_1
count,13899.0,13899.000000,13899.000000,13899.0,13899.0,13899.0,13017.000000,13899.0,13017.000000,13899.0,13899.0,13899.000000,13899.000000,13899.000000,13899.000000,13899.000000,13307.000000,13899.0,3957.000000,3957.000000,13017.0,13017.0,13017.0,13017.0,13017.0,13017.0,13017.0,13017.0,13017.0,13017.0,13017.0,13017.0,13017.0,13017.0,13017.0,13017.0,13017.0,13017.0,13899.0,13899.0,13899.0,13899.0,13899.0,13899.0,13899.0,13899.0,13899.0,13899.0,13899.0
mean,29.356285,252731.034607,7.122203,0.339953,1.025829,32.652637,34.315510,1.312756,2.106399,0.322685,18.550543,1.160102,11.824201,8.869117,61.929990,70.799108,645.544954,374.708252,3.174122,-14.639373,0.000077,0.056081,0.43036,0.858109,0.486287,1.058616,0.941461,1.449873,2.291542,1.409465,0.7945,1.890989,2.942076,2.845356,4.568641,2.985711,3.876162,4.673888,0.732067,0.032952,0.232391,0.00259,0.0,0.089575,0.063602,0.538384,0.255846,0.050004,0.00259
std,26.178106,81493.520396,9.213618,0.483035,17657.83387,35.116814,42.034887,1.224053,11.728047,1.588556,26.883052,0.020745,542.073929,23.685555,4329.930448,4330.846845,10727.185160,1301.554525,118.121759,137.674489,0.008765,0.631523,2.162589,3.976874,1.357722,2.856003,2.23726,2.942905,5.035405,2.620008,2.36542,3.641556,4.959798,4.589107,9.492132,5.965464,4.755245,6.136362,0.442898,0.178517,0.422372,0.050829,0.0,0.285582,0.244051,0.498542,0.436351,0.21796,0.050829
min,12.0,116589.000000,-16.000000,-1.0,-999999.0,0.0,-86.000000,1.0,-154.500000,0.0,0.0,1.112694,-18801.765600,-311.150000,-150414.124400,-150414.124400,0.000000,-41605.0,-122.000000,-7324.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,12.0,187356.000000,0.000000,0.0,0.0,10.0,8.000000,1.0,0.000000,0.0,0.0,1.145275,0.000000,0.000000,0.000000,0.000000,60.000000,269.0,-4.000000,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,13.0,226459.000000,4.000000,0.0,0.0,22.0,28.000000,1.0,0.000000,0.0,6.0,1.161131,0.000000,0.000000,0.000000,0.000000,223.000000,370.0,0.000000,6.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,2.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
75%,34.0,324063.000000,11.000000,1.0,2.0,45.0,52.000000,1.0,0.000000,0.0,31.0,1.172686,10.277750,7.200000,56.291750,64.223300,1000.000000,622.0,5.000000,13.000000,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2.0,3.0,2.0,0.0,2.0,4.0,4.0,5.0,4.0,6.0,6.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
max,83.0,485937.000000,221.000000,1.0,999999.0,715.0,1311.000000,45.0,756.000000,72.0,567.0,1.215136,34880.678900,414.130000,279045.431100,279045.431100,999999.990000,1735.0,7323.000000,48.000000,1.0,28.0,96.0,60.0,27.0,95.0,78.0,51.0,130.0,54.0,65.0,159.0,75.0,111.0,232.0,188.0,75.0,135.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0


In [36]:
orders_no_outliers.describe()


,site_id,owner_id,mains,order,covers,nb_saleslines,qty,payment_occurence,discount_amount,discount_occurence,account_discount_occurence,change_rate,tax_local,service_charge_local,revenue_excl_tax_local,tot_revenue_excl_tax_local,table_number,actual_duration,diff_seated_arrival_time,diff_opening_seated_time,Take_away_Bar,Breakfast,Extra_Bar,Extra_Food,Side,Beer,Wine_Bottle,Hot_Beverage,Alcohol,Mains_1,Wine_glass,Softs,Pizza,Desserts,Other,Cocktail,Pasta,Starters,Walkin,Group,Booking,Privatisation,CnC,AFTERNOON,LEMONADE,LUNCH,DINNER,BREAKFAST_1,PRIVATISATION_1
count,778013.0,778012.000000,778013.000000,778013.0,778013.0,778013.0,749180.000000,778013.0,749180.000000,778013.0,778013.0,778013.000000,778013.000000,778013.000000,778013.000000,778013.000000,7.769730e+05,778013.0,658168.000000,658170.000000,749180.0,749180.0,749180.0,749180.0,749180.0,749180.0,749180.0,749180.0,749180.0,749180.0,749180.0,749180.0,749180.0,749180.0,749180.0,749180.0,749180.0,749180.0,778013.0,778013.0,778013.0,778013.0,778013.0,778013.0,778013.0,778013.0,778013.0,778013.0,778013.0
mean,28.203448,269169.984554,2.524212,0.950289,2.650822,9.94412,9.797263,1.49142,1.326679,0.173725,0.288931,1.161869,16.321990,12.655850,86.910373,99.566224,1.097217e+02,84.191088,0.997806,7.982280,0.000019,0.030457,0.085221,0.128471,0.15612,0.277016,0.312492,0.312632,0.323097,0.412559,0.453854,0.644394,0.646101,0.905864,0.885703,1.175481,1.543872,1.652,0.314801,0.008905,0.67571,0.000577,0.000008,0.069247,0.014513,0.34606,0.558243,0.011361,0.000577
std,22.780481,79276.686330,1.887077,0.255481,3.92597,7.708381,10.282937,1.054752,6.534227,0.830741,3.022473,0.021191,21.631733,11.965471,110.227470,118.950395,4.704447e+03,38.369779,283.400952,283.313169,0.005167,0.326976,0.512511,0.492455,0.519602,0.817032,0.79149,0.906356,1.077535,0.868176,0.999673,1.033198,0.997278,1.454889,2.197602,1.807511,1.301842,1.882815,0.464437,0.093944,0.468109,0.024016,0.002777,0.253874,0.119591,0.475713,0.496597,0.105981,0.024016
min,12.0,147145.000000,-87.000000,-1.0,-991.0,0.0,-837.000000,1.0,-324.000000,0.0,0.0,1.112694,-1060.580000,-414.130000,-5302.920000,-5302.920000,0.000000e+00,-39.0,-584.000000,-203465.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,13.0,207205.000000,2.000000,1.0,2.0,5.0,6.000000,1.0,0.000000,0.0,0.0,1.146251,8.666700,7.130000,49.166600,56.079900,2.300000e+01,59.0,-4.000000,4.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,13.0,261097.000000,2.000000,1.0,2.0,8.0,8.000000,1.0,0.000000,0.0,0.0,1.162853,13.166700,10.440000,70.833300,81.121100,5.300000e+01,80.0,0.000000,9.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
75%,34.0,330294.000000,3.000000,1.0,3.0,12.0,12.000000,2.0,0.000000,0.0,0.0,1.176407,19.889000,15.500000,104.583400,120.093300,1.200000e+02,105.0,5.000000,13.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
max,83.0,485937.000000,230.000000,1.0,991.0,401.0,5018.000000,29.0,625.500000,143.0,437.0,1.215136,4502.200000,795.750000,22511.000000,22511.000000,4.114070e+06,233.0,203478.000000,77.000000,2.0,22.0,48.0,30.0,25.0,52.0,47.0,42.0,57.0,51.0,80.0,41.0,75.0,163.0,218.0,146.0,44.0,90.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


#### Z-score on covers column

In [59]:
# Calculate Z-scores for each numerical column
z_scores = np.abs((orders_no_outliers['covers'] - orders_no_outliers['covers'].mean()) / orders_no_outliers['covers'].std())

# Define a threshold for outliers (e.g., Z-score > 3, meaning values beyond 3 standard deviations)
threshold =2.5

# Create a boolean mask for outliers
outlier_mask = (z_scores > threshold)

# Remove outliers from the DataFrame
orders_no_outliers2 = orders_no_outliers[~outlier_mask]



In [49]:
a=(233-2.647313)/2.5324
a

90.96220462802086

In [60]:
orders_no_outliers2.describe()

,site_id,owner_id,mains,order,covers,nb_saleslines,qty,payment_occurence,discount_amount,discount_occurence,account_discount_occurence,change_rate,tax_local,service_charge_local,revenue_excl_tax_local,tot_revenue_excl_tax_local,table_number,actual_duration,diff_seated_arrival_time,diff_opening_seated_time,Take_away_Bar,Breakfast,Extra_Bar,Extra_Food,Side,Beer,Wine_Bottle,Hot_Beverage,Alcohol,Mains_1,Wine_glass,Softs,Pizza,Desserts,Other,Cocktail,Pasta,Starters,Walkin,Group,Booking,Privatisation,CnC,AFTERNOON,LEMONADE,LUNCH,DINNER,BREAKFAST_1,PRIVATISATION_1
count,770972.0,770971.000000,770972.000000,770972.0,770972.0,770972.0,742331.000000,770972.0,742331.000000,770972.0,770972.0,770972.000000,770972.000000,770972.000000,770972.000000,770972.000000,7.699320e+05,770972.0,656994.000000,656996.000000,742331.0,742331.0,742331.0,742331.0,742331.0,742331.0,742331.0,742331.0,742331.0,742331.0,742331.0,742331.0,742331.0,742331.0,742331.0,742331.0,742331.0,742331.0,770972.0,770972.0,770972.0,770972.0,770972.0,770972.0,770972.0,770972.0,770972.0,770972.0,770972.0
mean,28.24186,269126.178296,2.544172,0.954815,2.599292,9.875257,9.739064,1.47853,1.324530,0.17265,0.287985,1.161886,15.834557,12.287445,84.454304,96.741750,1.017819e+02,84.042602,0.996801,7.985924,0.000019,0.030567,0.081516,0.128576,0.156174,0.269125,0.298313,0.310704,0.317088,0.412036,0.45192,0.637765,0.646945,0.908141,0.880979,1.151519,1.547485,1.655097,0.314625,0.004287,0.680866,0.000214,0.000008,0.069363,0.014519,0.346983,0.557499,0.011422,0.000214
std,22.807985,79304.274296,1.837564,0.237397,1.379387,7.595236,9.986437,1.000654,6.366604,0.825565,3.002996,0.021188,19.518464,9.555356,99.507345,105.649588,4.724160e+03,37.830508,283.653511,283.563321,0.005191,0.326903,0.493192,0.492056,0.519146,0.785014,0.745534,0.894315,1.059067,0.864507,0.994461,1.017773,0.995099,1.452721,2.181996,1.73509,1.295197,1.876166,0.464367,0.065333,0.466141,0.014628,0.00279,0.254071,0.119618,0.476011,0.496683,0.106261,0.014628
min,12.0,147145.000000,-87.000000,-1.0,-3.0,0.0,-837.000000,1.0,-180.000000,0.0,0.0,1.112694,-1060.580000,-414.130000,-5302.920000,-5302.920000,0.000000e+00,-39.0,-584.000000,-203465.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,13.0,207205.000000,2.000000,1.0,2.0,5.0,6.000000,1.0,0.000000,0.0,0.0,1.146282,8.666800,7.130000,49.166600,56.080000,2.300000e+01,59.0,-4.000000,4.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,13.0,261097.000000,2.000000,1.0,2.0,8.0,8.000000,1.0,0.000000,0.0,0.0,1.162870,13.166600,10.390000,70.666600,80.986700,5.300000e+01,80.0,0.000000,9.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
75%,34.0,330294.000000,3.000000,1.0,3.0,12.0,12.000000,2.0,0.000000,0.0,0.0,1.176407,19.749900,15.380000,103.750001,119.226899,1.200000e+02,105.0,5.000000,13.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
max,83.0,485937.000000,230.000000,1.0,8.0,401.0,5018.000000,20.0,625.500000,143.0,437.0,1.215136,4502.200000,795.750000,22511.000000,22511.000000,4.114070e+06,233.0,203478.000000,77.000000,2.0,22.0,48.0,30.0,25.0,52.0,47.0,42.0,57.0,51.0,80.0,41.0,75.0,163.0,218.0,146.0,44.0,90.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [63]:
orders=orders_no_outliers2
orders.describe()

,site_id,owner_id,mains,order,covers,nb_saleslines,qty,payment_occurence,discount_amount,discount_occurence,account_discount_occurence,change_rate,tax_local,service_charge_local,revenue_excl_tax_local,tot_revenue_excl_tax_local,table_number,actual_duration,diff_seated_arrival_time,diff_opening_seated_time,Take_away_Bar,Breakfast,Extra_Bar,Extra_Food,Side,Beer,Wine_Bottle,Hot_Beverage,Alcohol,Mains_1,Wine_glass,Softs,Pizza,Desserts,Other,Cocktail,Pasta,Starters,Walkin,Group,Booking,Privatisation,CnC,AFTERNOON,LEMONADE,LUNCH,DINNER,BREAKFAST_1,PRIVATISATION_1
count,770972.0,770971.000000,770972.000000,770972.0,770972.0,770972.0,742331.000000,770972.0,742331.000000,770972.0,770972.0,770972.000000,770972.000000,770972.000000,770972.000000,770972.000000,7.699320e+05,770972.0,656994.000000,656996.000000,742331.0,742331.0,742331.0,742331.0,742331.0,742331.0,742331.0,742331.0,742331.0,742331.0,742331.0,742331.0,742331.0,742331.0,742331.0,742331.0,742331.0,742331.0,770972.0,770972.0,770972.0,770972.0,770972.0,770972.0,770972.0,770972.0,770972.0,770972.0,770972.0
mean,28.24186,269126.178296,2.544172,0.954815,2.599292,9.875257,9.739064,1.47853,1.324530,0.17265,0.287985,1.161886,15.834557,12.287445,84.454304,96.741750,1.017819e+02,84.042602,0.996801,7.985924,0.000019,0.030567,0.081516,0.128576,0.156174,0.269125,0.298313,0.310704,0.317088,0.412036,0.45192,0.637765,0.646945,0.908141,0.880979,1.151519,1.547485,1.655097,0.314625,0.004287,0.680866,0.000214,0.000008,0.069363,0.014519,0.346983,0.557499,0.011422,0.000214
std,22.807985,79304.274296,1.837564,0.237397,1.379387,7.595236,9.986437,1.000654,6.366604,0.825565,3.002996,0.021188,19.518464,9.555356,99.507345,105.649588,4.724160e+03,37.830508,283.653511,283.563321,0.005191,0.326903,0.493192,0.492056,0.519146,0.785014,0.745534,0.894315,1.059067,0.864507,0.994461,1.017773,0.995099,1.452721,2.181996,1.73509,1.295197,1.876166,0.464367,0.065333,0.466141,0.014628,0.00279,0.254071,0.119618,0.476011,0.496683,0.106261,0.014628
min,12.0,147145.000000,-87.000000,-1.0,-3.0,0.0,-837.000000,1.0,-180.000000,0.0,0.0,1.112694,-1060.580000,-414.130000,-5302.920000,-5302.920000,0.000000e+00,-39.0,-584.000000,-203465.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,13.0,207205.000000,2.000000,1.0,2.0,5.0,6.000000,1.0,0.000000,0.0,0.0,1.146282,8.666800,7.130000,49.166600,56.080000,2.300000e+01,59.0,-4.000000,4.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,13.0,261097.000000,2.000000,1.0,2.0,8.0,8.000000,1.0,0.000000,0.0,0.0,1.162870,13.166600,10.390000,70.666600,80.986700,5.300000e+01,80.0,0.000000,9.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
75%,34.0,330294.000000,3.000000,1.0,3.0,12.0,12.000000,2.0,0.000000,0.0,0.0,1.176407,19.749900,15.380000,103.750001,119.226899,1.200000e+02,105.0,5.000000,13.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
max,83.0,485937.000000,230.000000,1.0,8.0,401.0,5018.000000,20.0,625.500000,143.0,437.0,1.215136,4502.200000,795.750000,22511.000000,22511.000000,4.114070e+06,233.0,203478.000000,77.000000,2.0,22.0,48.0,30.0,25.0,52.0,47.0,42.0,57.0,51.0,80.0,41.0,75.0,163.0,218.0,146.0,44.0,90.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [67]:
orders.to_csv('orders_no_outliers.csv',index=False)